<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=ff675ea0146275e7db27390b21b0ce5db8510f9342cc1ebabdb4ca71e8ad832c
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-26 10:40:42
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16


-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.37 C
-------------------
Today PnL: -1.21 L (-0.88%)
Current PnL: -22.26 L (-14.97%)
CY Booked + Current PnL: -10.82 L (-7.28%)
-------------------
Total profit:  96.05 K
Total loss:  -23.22 L
-------------------
Total Booked + Current PnL: 16.16 L (13.28%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.33%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 88.13 L (64.17%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.33%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,11.32,52.0,H-MC,3.87,192090.0,27420.0,15905.0,-0.33,16.65,8.28,26.31,89.0,1.72,1.40,25.96,X40N,NTT,AC
77,TTKPRESTIG,770.00,94.66,38.0,M-SC,2.77,83912.0,-16865.0,16959.0,-1.94,-16.73,20.21,0.09,245.0,-0.99,0.61,10.24,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-46.15,48.0,H-MC,10.91,106822.0,6318.0,22785.0,-1.97,6.29,21.33,28.96,79.0,0.28,0.78,42.39,MH,ATH,METALS
42,ITC,452.00,-39.19,41.0,H-LC,2.18,196578.0,-3560.0,24454.0,0.47,-1.78,12.44,10.44,4.0,-0.15,1.43,3.69,X40,NTT,FMCG
57,RELIANCE,1533.00,-14.41,42.0,H-LC,4.06,214297.0,3931.0,24858.0,0.09,1.87,11.60,13.68,37.0,0.16,1.56,18.70,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TATAMOTORS,1065.00,-22.16,43.0,M-LC,2.36,245715.0,-30808.0,140868.0,1.90,-11.14,57.33,39.81,54.0,-0.22,1.79,17.76,XY24,NTT,AUTO
34,ICICIGI,2260.25,-18.02,54.0,H-MC,1.88,140844.0,4711.0,26408.0,1.14,3.46,18.75,22.86,91.0,0.18,1.03,16.51,X40,ATH,INSURANCE
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,5.89,82770.0,-19440.0,112832.0,1.13,-19.02,136.32,91.37,208.0,-0.17,0.60,34.36,XY25,NTT,FINANCE
64,SHALBY,327.00,1191.87,67.0,M-SC,12.79,175759.0,-5384.0,50847.0,1.05,-2.97,28.93,25.10,235.0,-0.11,1.28,40.85,XY24,NTT,HEALTHCARE
55,RAJOOENG,143.10,-41.63,27.0,H-SC,13.31,88450.0,-14050.0,54653.0,0.69,-13.71,61.79,39.61,114.0,-0.26,0.64,0.69,AR,ATH,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,HAPPSTMNDS,1488.71,-20.32,27.0,H-SC,10.52,85920.0,-42245.0,152276.0,-2.90,-32.96,177.23,85.85,132.0,-0.28,0.63,0.00,AR,ATH,IT
46,KPIGREEN,731.05,-3.26,32.0,H-SC,8.77,115403.0,-9894.0,68826.0,-2.68,-7.90,59.64,47.03,141.0,-0.14,0.84,44.50,MH,ATH,POWER
32,HINDZINC,730.22,21.48,55.0,M-LC,2.80,198077.0,-6999.0,119559.0,-2.44,-3.41,60.36,54.89,52.0,-0.06,1.44,19.86,X5K,ATH,METALS
56,RELAXO,1176.00,-41.23,31.0,H-SC,2.10,80163.0,-64997.0,140927.0,-2.29,-44.78,175.80,52.31,136.0,-0.46,0.58,7.30,X40N,NTT,FOOTWEAR
54,RAJESHEXPO,518.00,1695.45,41.0,L-SC,2.60,48993.0,-88184.0,88276.0,-2.19,-64.28,180.18,0.07,267.0,-1.00,0.36,21.85,OX40N,NTT,JEWELLERY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,92.53,49.0,H-SC,10.19,121257.0,-21222.0,40742.0,-0.13,-14.89,33.60,13.70,163.0,-0.52,0.88,43.12,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,94.66,38.0,M-SC,2.77,83912.0,-16865.0,16959.0,-1.94,-16.73,20.21,0.09,245.0,-0.99,0.61,10.24,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-68.16,47.0,H-SC,1.39,219051.0,-50616.0,86941.0,-0.75,-18.77,39.69,13.47,138.0,-0.58,1.60,10.79,XY24,NTT,PAINTS
18,CERA,9475.0,-22.67,34.0,H-SC,1.85,140990.0,-34913.0,76938.0,0.03,-19.85,54.57,23.89,149.0,-0.45,1.03,21.76,OX40N,NTT,CERAMICS
66,SIS,528.0,2075.65,39.0,H-SC,4.22,87796.0,-23236.0,46839.0,-0.52,-20.93,53.35,21.26,156.0,-0.50,0.64,18.62,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.95,59.0,M-MC,10.30,230748.0,5786.0,161870.0,-1.85,2.57,70.15,74.53,192.0,0.04,1.68,35.65,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,11.32,52.0,H-MC,3.87,192090.0,27420.0,15905.0,-0.33,16.65,8.28,26.31,89.0,1.72,1.40,25.96,X40N,NTT,AC
84,VOLTAS,1530.00,-3.18,41.0,H-MC,3.04,203025.0,11283.0,26474.0,-1.38,5.88,13.04,19.69,99.0,0.43,1.48,13.08,XY25,NTT,AC
17,CAMS,5211.76,-5.61,45.0,H-SC,1.76,107428.0,5424.0,38502.0,-0.82,5.32,35.84,43.06,122.0,0.14,0.78,23.34,X40N,ATH,MISC
79,UNIONBANK,163.00,-14.73,54.0,M-LC,6.83,154916.0,14076.0,30410.0,-1.82,9.99,19.63,31.59,66.0,0.46,1.13,39.13,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-46.15,48.0,H-MC,10.91,106822.0,6318.0,22785.0,-1.97,6.29,21.33,28.96,79.0,0.28,0.78,42.39,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-44.45,33.0,M-SC,1.50,93825.0,2326.0,76420.0,0.16,2.54,81.45,86.07,223.0,0.03,0.68,36.25,XR,NTT,DURABLES
11,BANKINDIA,190.00,-29.19,56.0,H-MC,8.24,182514.0,2706.0,109709.0,-0.91,1.51,60.11,62.52,88.0,0.02,1.33,36.11,XR,NTT,BANKS
37,INDIAMART,4850.92,-55.20,25.0,H-SC,10.23,124974.0,1638.0,132123.0,-0.66,1.33,105.72,108.45,119.0,0.01,0.91,23.98,AR,ATH,MISC
86,WIPRO,420.00,-16.06,34.0,M-LC,5.64,147269.0,-3676.0,113132.0,-1.85,-2.44,76.82,72.51,53.0,-0.03,1.07,3.44,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-17.77,29.0,H-LC,7.03,207530.0,-44238.0,97456.0,-1.90,-17.57,46.96,21.13,27.0,-0.45,1.51,11.63,X40,ATH,PAINTS
44,JIOFIN,387.00,-5.12,30.0,H-LC,12.97,257828.0,-9537.0,75002.0,-0.60,-3.57,29.09,24.48,48.0,-0.13,1.88,49.44,XY24,BTT,FINANCE
74,TCS,4476.75,-30.93,31.0,H-LC,12.06,275523.0,-70433.0,145283.0,-0.89,-20.36,52.73,21.64,1.0,-0.48,2.01,0.00,X40,ATH,IT
83,VBL,671.64,-20.19,32.0,H-LC,8.17,287387.0,-28455.0,141107.0,-0.31,-9.01,49.10,35.67,5.0,-0.20,2.09,3.60,X40N,ATH,FMCG
30,HCLTECH,1943.91,-3.23,36.0,H-LC,9.91,220962.0,-20962.0,84231.0,-1.15,-8.66,38.12,26.15,8.0,-0.25,1.61,4.42,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-6.94,41.0,H-LC,1.69,230580.0,-15571.0,94768.0,-1.73,-6.33,41.10,32.18,16.0,-0.16,1.68,27.44,X200,ATH,IT
42,ITC,452.0,-39.19,41.0,H-LC,2.18,196578.0,-3560.0,24454.0,0.47,-1.78,12.44,10.44,4.0,-0.15,1.43,3.69,X40,NTT,FMCG
65,SIEMENS,4671.5,-2.61,43.0,H-LC,2.61,156410.0,-29685.0,77173.0,-0.89,-15.95,49.34,25.51,15.0,-0.38,1.14,15.26,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.41,42.0,H-LC,4.06,214297.0,3931.0,24858.0,0.09,1.87,11.60,13.68,37.0,0.16,1.56,18.70,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-11.41,43.0,H-LC,4.34,272821.0,7395.0,49408.0,-0.65,2.79,18.11,21.40,11.0,0.15,1.99,10.32,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-30.93,31.0,H-LC,12.06,275523.0,-70433.0,145283.0,-0.89,-20.36,52.73,21.64,1.0,-0.48,2.01,0.00,X40,ATH,IT
80,UNITDSPR,1644.00,-14.09,45.0,H-LC,7.15,225303.0,-8841.0,57475.0,-0.58,-3.78,25.51,20.77,86.0,-0.15,1.64,2.88,X40N,NTT,BREWERIES
83,VBL,671.64,-20.19,32.0,H-LC,8.17,287387.0,-28455.0,141107.0,-0.31,-9.01,49.10,35.67,5.0,-0.20,2.09,3.60,X40N,ATH,FMCG
42,ITC,452.00,-39.19,41.0,H-LC,2.18,196578.0,-3560.0,24454.0,0.47,-1.78,12.44,10.44,4.0,-0.15,1.43,3.69,X40,NTT,FMCG
30,HCLTECH,1943.91,-3.23,36.0,H-LC,9.91,220962.0,-20962.0,84231.0,-1.15,-8.66,38.12,26.15,8.0,-0.25,1.61,4.42,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.61,43.0,H-LC,2.61,156410.0,-29685.0,77173.0,-0.89,-15.95,49.34,25.51,15.0,-0.38,1.14,15.26,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,41.0,H-LC,2.18,196578.0,-3560.0,24454.0,0.47,-1.78,12.44,10.44,4.0,-0.15,1.43,3.69,X40,NTT,FMCG
19,CIPLA,1795.00,-23.21,36.0,H-LC,4.55,205062.0,562.0,40848.0,-0.85,0.27,19.92,20.25,10.0,0.01,1.49,9.03,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-17.77,29.0,H-LC,7.03,207530.0,-44238.0,97456.0,-1.90,-17.57,46.96,21.13,27.0,-0.45,1.51,11.63,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.41,42.0,H-LC,4.06,214297.0,3931.0,24858.0,0.09,1.87,11.60,13.68,37.0,0.16,1.56,18.70,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-30.93,31.0,H-LC,12.06,275523.0,-70433.0,145283.0,-0.89,-20.36,52.73,21.64,1.0,-0.48,2.01,0.00,X40,ATH,IT
40,INFY,2275.00,-19.96,42.0,H-LC,8.12,311938.0,-950.0,172626.0,-1.37,-0.30,55.34,54.87,3.0,-0.01,2.27,6.33,X40,BTT,IT
42,ITC,452.00,-39.19,41.0,H-LC,2.18,196578.0,-3560.0,24454.0,0.47,-1.78,12.44,10.44,4.0,-0.15,1.43,3.69,X40,NTT,FMCG
83,VBL,671.64,-20.19,32.0,H-LC,8.17,287387.0,-28455.0,141107.0,-0.31,-9.01,49.10,35.67,5.0,-0.20,2.09,3.60,X40N,ATH,FMCG
1,ABB,7934.00,-40.84,41.0,H-LC,7.98,247046.0,-14573.0,133775.0,-1.04,-5.57,54.15,45.57,7.0,-0.11,1.80,5.12,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,43.0,L-SC,41.40,79584.0,-33965.0,74013.0,-2.17,-29.91,93.00,35.27,268.0,-0.46,0.58,94.78,XR,NTT,HOTELS
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,5.89,82770.0,-19440.0,112832.0,1.13,-19.02,136.32,91.37,208.0,-0.17,0.60,34.36,XY25,NTT,FINANCE
49,MASFIN,398.61,-19.15,41.0,H-SC,12.54,92010.0,-5970.0,27575.0,-0.68,-6.09,29.97,22.05,152.0,-0.22,0.67,33.74,XR,ATH,FINANCE
50,NATIONALUM,244.55,-46.15,48.0,H-MC,10.91,106822.0,6318.0,22785.0,-1.97,6.29,21.33,28.96,79.0,0.28,0.78,42.39,MH,ATH,METALS
78,UJJIVANSFB,60.00,92.53,49.0,H-SC,10.19,121257.0,-21222.0,40742.0,-0.13,-14.89,33.60,13.70,163.0,-0.52,0.88,43.12,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1191.87,67.0,M-SC,12.79,175759.0,-5384.0,50847.0,1.05,-2.97,28.93,25.10,235.0,-0.11,1.28,40.85,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,5.89,82770.0,-19440.0,112832.0,1.13,-19.02,136.32,91.37,208.0,-0.17,0.60,34.36,XY25,NTT,FINANCE
71,TANLA,1963.11,-18.28,59.0,H-SC,16.88,218729.0,-52032.0,381988.0,-0.81,-19.22,174.64,121.86,133.0,-0.14,1.59,71.11,AR,ATH,IT
81,VAIBHAVGBL,521.00,55.37,53.0,H-SC,5.60,134559.0,-48216.0,159291.0,-1.01,-26.38,118.38,60.77,125.0,-0.30,0.98,20.52,XR,NTT,JEWELLERY
7,ATULAUTO,844.00,3629.73,50.0,M-SC,7.15,116218.0,-31150.0,82968.0,0.15,-21.14,71.39,35.16,236.0,-0.38,0.85,18.71,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.25
1,25,43.72
2,50,74.90


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.92
LC    32.41
MC    23.67
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.37
X40      14.75
X40N     11.40
XR       11.16
XY25     10.13
AR        8.33
OX40N     7.90
X200      1.68
MH        1.62
X5K       1.44
SR        1.22
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.38
H-LC    25.93
H-MC    20.71
M-SC    13.07
M-LC     5.43
M-MC     2.66
L-SC     1.47
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.02,-6.65,41.86
IT,12.86,-19.05,83.96
FINANCE,10.07,-18.63,70.39
MISC,7.00,-19.47,85.88
BANKS,6.15,-18.06,80.17
PAINTS,5.65,-19.12,36.87
ELECTRICAL,5.47,-9.85,48.39
INSURANCE,3.77,-6.81,46.73
AC,3.59,1.65,14.81


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3212990.0,22
XR,1378030.0,14
AR,1188205.0,9
X40,823886.0,10
X40N,639001.0,9
OX40N,562280.0,10
XY25,448152.0,7
SR,254031.0,2
X5K,119559.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3759183.0,29
M-SC,1450912.0,17
H-MC,1265004.0,15
H-LC,1259764.0,15
M-LC,403969.0,4
M-MC,312619.0,2
L-SC,260215.0,3
L-MC,61265.0,1
L-LC,39582.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1252241.0      6
M-SC       XY24       829239.0      7
H-SC       AR         807570.0      5
           XR         797770.0      7
H-MC       XY24       572611.0      4
H-LC       X40        524050.0      5
H-SC       X40N       315732.0      4
M-MC       XY24       312619.0      2
H-SC       OX40N      263013.0      4
           SR         254031.0      2
H-LC       X40N       239430.0      3
H-MC       X40        223739.0      4
H-LC       AR         210948.0      2
H-MC       XY25       180172.0      2
L-SC       XR         171939.0      2
M-LC       XY24       171278.0      2
M-SC       AR         169687.0      2
           OX40N      138842.0      4
           XR         124215.0      2
M-LC       X5K        119559.0      1
H-LC       XY25       115566.0      3
M-LC       XR         113132.0      1
M-SC       XY25       112832.0      1
H-MC       XR         109709.0      1
H-LC       X200        94768.0      1
L-SC       OX40N       88276.0      1
H-MC       X40N        83839.0      2
M-SC       X40         76097.0      1
H-LC       XY24        75002.0      1
H-MC       OX40N       72149.0      1
H-SC       MH          68826.0      1
L-MC       XR          61265.0      1
L-LC       XY25        39582.0      1
H-MC       MH          22785.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 43.0 seconds
